In [59]:
from sklearn import datasets as DS
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz as EG
import pydotplus as PP
import math

In [60]:
def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [61]:
iris=DS.load_iris()
df=pd.DataFrame(iris.data,columns=iris.feature_names)
y=iris.target
                                    #Loading of iris dataset

In [62]:
df['sepal length (cm)'] = toLabel(df, 'sepal length (cm)')
df['sepal width (cm)'] = toLabel(df, 'sepal width (cm)')
df['petal length (cm)'] = toLabel(df, 'petal length (cm)')
df['petal width (cm)'] = toLabel(df, 'petal width (cm)')

                                                        #Converting pandas data directly into labelled data

In [63]:
df.columns=[1,2,3,4]
COLUMN_NAMESS={}
COLUMN_NAMESS[1]=iris.feature_names[0]
COLUMN_NAMESS[2]=iris.feature_names[1]
COLUMN_NAMESS[3]=iris.feature_names[2]
COLUMN_NAMESS[4]=iris.feature_names[3]

                    #A dictionary initialization for storing column/feature names and number assigned to them

In [64]:
df[5]=y                                               #Appending the class type(eg. Setosa,Virginica ) column to the pandas loaded before

CLASS_NAMESS=df[df.columns.values[-1]].unique()       #CLASS_NAMESS is an array for the 3 values of class type(eg. setosa,virginica )

total_keys=df.columns.values                #Total Keys holds the number equivalent of feature/column names (eg. 0 for setosa, 1 for versicolor)                             

CLASS_NAMESS

array([0, 1, 2], dtype=int64)

In [65]:
def func(df,ent_par,lvl):           #ent_par is the entropy of current node/label passed to find information gain and hence,split further
    
    total_keys=df.columns.values
    
    if(ent_par==0):
        print("Level ",lvl)
        print("Count of ",CLASS_NAMESS[0],"(",iris.target_names[CLASS_NAMESS[0]],") = ",df.loc[df[total_keys[-1]]==CLASS_NAMESS[0]].shape[0])
        print("Count of ",CLASS_NAMESS[1],"(",iris.target_names[CLASS_NAMESS[1]],") = ",df.loc[df[total_keys[-1]]==CLASS_NAMESS[1]].shape[0])
        print("Count of ",CLASS_NAMESS[2],"(",iris.target_names[CLASS_NAMESS[2]],") = ",df.loc[df[total_keys[-1]]==CLASS_NAMESS[2]].shape[0])
        print("Current Entropy is = ",ent_par)
        print("Reached leaf Node")
        print()
        print()                                     
        return                                         #Print statement when the recursion reaches leaf node
     
    if len(total_keys)==1:
        print("Level ",lvl)
        print("Count of ",CLASS_NAMESS[0],"(",iris.target_names[CLASS_NAMESS[0]],") = ",df.loc[df[total_keys[-1]]==CLASS_NAMESS[0]].shape[0])
        print("Count of ",CLASS_NAMESS[1],"(",iris.target_names[CLASS_NAMESS[1]],") = ",df.loc[df[total_keys[-1]]==CLASS_NAMESS[1]].shape[0])
        print("Count of ",CLASS_NAMESS[2],"(",iris.target_names[CLASS_NAMESS[2]],") = ",df.loc[df[total_keys[-1]]==CLASS_NAMESS[2]].shape[0])
        print("Current Entropy is = ",ent_par)
        print("No Feature Left")
        print()
        print()                                     
        return     
    
    feat_cons=-1
    
    ent_change=0                                       #Maximum Information Gain Ratio
    dic_for_en_i={}                         #Dictionary for storing entropy for different values of chosen feature 
    
    for i in total_keys:
        if (i == total_keys[-1]):
            continue
        
        uniq_val_i=df[i].unique()           #Distinct values available to split upon for the current feature
        
        uniq_val_class=df[total_keys[-1]].unique()       #Distinct values of classes available in the current copy of pandas passed in recursion
        
        Sno=0                               #Split number for current feature
        
        en_hold = {}                        #Array to store entropy of different labels of current feature
        en_total = 0                        #Total entropy ,if we choose to split on the current feature
        
        for j in uniq_val_i:              
            
            temp1=df.loc[df[i]==j].copy()   #temp1 is just to avoid rebundant code 'df.loc[......]' every time we desire to iterate upon a particular column'
            temp=temp1.shape[0]
            temp/=df.shape[0]
            temp*=math.log10(temp)
            
            Sno-=temp                       # Subtracting (Di/D) * log(Di / D)
            
            Ptotal=temp1.shape[0]/df.shape[0]   #Ptotal = (Di/D)
            
            en_j = 0                             #Entropy for current label split
            
            for k in uniq_val_class:
                temp3 = temp1.loc[temp1[total_keys[-1]]==k].shape[0]
                
                if(temp3==0):
                    continue
                    
                temp3 /= temp1.shape[0]
                
                temp3*=math.log10(temp3)
                en_j-=temp3
            
            en_hold[j] = en_j                 #Updating en_hold with entropy of current label
            en_j*=Ptotal
            
            en_total+=en_j
        
        en_change = (-en_total + ent_par) / Sno
        
        if en_change >= ent_change:         #Checking current gain calculated greater than gain ratio calculated till now?
            ent_change=en_change
            feat_cons=i
            dic_for_en_i.clear()
            dic_for_en_i = en_hold          #If yes , change the dictionary for the current featre,that is, storing entropy for all the labels of splitting feature
    
    uniq_val_i=df[feat_cons].unique()       #Finally we split below on the feature considered with maximum gain raio : 

    print("Level ",lvl)
    print("Count of ",CLASS_NAMESS[0],"(",iris.target_names[CLASS_NAMESS[0]],") = ",df.loc[df[total_keys[-1]]==CLASS_NAMESS[0]].shape[0])
    print("Count of ",CLASS_NAMESS[1],"(",iris.target_names[CLASS_NAMESS[1]],") = ",df.loc[df[total_keys[-1]]==CLASS_NAMESS[1]].shape[0])
    print("Count of ",CLASS_NAMESS[2],"(",iris.target_names[CLASS_NAMESS[2]],") = ",df.loc[df[total_keys[-1]]==CLASS_NAMESS[2]].shape[0])
       
    print("Current Entropy is = ",ent_par)
    print("Splitting on feature ",COLUMN_NAMESS[feat_cons]," with gain ratio ",en_change)
    print()
    print()                                     #Print statement / Output for chosen feature
    
    for i in uniq_val_i:                        #Loop to call recursive split on all labels chosen feature, each having it's own modified copy of pandas
        tempo=df.loc[df[feat_cons]==i].copy()
        tempo.drop([feat_cons],axis=1,inplace = True)
        func(tempo,dic_for_en_i[i],lvl+1)
        

In [66]:
c_names=df.columns.values
unique_val_class=df[c_names[-1]].unique()
total=df.shape[0]

curr_en=0

for i in unique_val_class:
    temp=df.loc[df[c_names[-1]]==i].shape[0]
    temp/=total
    
    temp*=math.log10(temp)
    
    curr_en-=temp
                            #Calculating the current entropy for base case to begin dividing on features in recursion
    
func(df.copy(),curr_en,0)   #At every recursive step, a modified copy of pandas is passed for future operations

Level  0
Count of  0 ( setosa ) =  50
Count of  1 ( versicolor ) =  50
Count of  2 ( virginica ) =  50
Current Entropy is =  0.4771212547196624
Splitting on feature  petal width (cm)  with gain ratio  0.6996382036222091


Level  1
Count of  0 ( setosa ) =  50
Count of  1 ( versicolor ) =  0
Count of  2 ( virginica ) =  0
Current Entropy is =  0.0
Reached leaf Node


Level  1
Count of  0 ( setosa ) =  0
Count of  1 ( versicolor ) =  40
Count of  2 ( virginica ) =  16
Current Entropy is =  0.2598251810131059
Splitting on feature  petal length (cm)  with gain ratio  0.43340994956210654


Level  2
Count of  0 ( setosa ) =  0
Count of  1 ( versicolor ) =  39
Count of  2 ( virginica ) =  8
Current Entropy is =  0.1981353138938235
Splitting on feature  sepal length (cm)  with gain ratio  0.04553496474578599


Level  3
Count of  0 ( setosa ) =  0
Count of  1 ( versicolor ) =  2
Count of  2 ( virginica ) =  0
Current Entropy is =  0.0
Reached leaf Node


Level  3
Count of  0 ( setosa ) =  0
Cou